In [370]:
NUM_STOCKS=2
import yahoo_fin as yf
from yahoo_fin.stock_info import get_data
import time
import datetime
import pandas as pd
from yahoo_fin import stock_info as si

import tensorflow as tf 
import numpy as np
from tensorflow.keras import layers

import concurrent.futures

# getting data

things we did to clean data:

1. shamelessly copy:
https://levelup.gitconnected.com/how-to-get-all-stock-symbols-a73925c16a1b

2. we removed the tickers with $

3. we added exception handeling that tries similer strings to the input (note that this is not implemented on the set itself but when fetching, leading to multiple requsts and potentialy to errors)
4.  we just drop exceptions that are found when getting the data from online  (these are diffrent every day)

# how are we keeping consistency? 
since the avilble stocks may change depending on the source of the data some extra steps need to be done in order to
alow saves to work seemlesly 


basic plan:

1. clean the data as much as possible 

2. save the embedings weights with their ticker string

3. backup everything on git to alow us to fix it



In [2]:
df1 = pd.DataFrame( si.tickers_sp500() )
df2 = pd.DataFrame( si.tickers_nasdaq() )
df3 = pd.DataFrame( si.tickers_dow() )
df4 = pd.DataFrame( si.tickers_other() )

sym1 = set( symbol for symbol in df1[0].values.tolist() )
sym2 = set( symbol for symbol in df2[0].values.tolist() )
sym3 = set( symbol for symbol in df3[0].values.tolist() )
sym4 = set( symbol for symbol in df4[0].values.tolist() )

symbols = set.union( sym1, sym2, sym3, sym4 )
del(sym1, sym2, sym3, sym4,df1, df2, df3, df4)

In [3]:
my_list = ['W', 'R', 'P', 'Q']
del_set = set()
sav_set = set()

for symbol in symbols:
    if (len( symbol ) > 4 and symbol[-1] in my_list or "$" in symbol):
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
sav_set.discard("") 

ticker_list=list(sav_set)

print( f'Removed {len( del_set )} unqualified stock symbols...' )
print( f'There are {len( sav_set )} qualified stock symbols...' )

Removed 1584 unqualified stock symbols...
There are 10628 qualified stock symbols...


In [4]:
start='01/01/2020' 
end='09/14/2022'

In [5]:
import concurrent.futures 

def get_seiries(ticker,start,end,entry="adjclose"):
    try:
        raw = get_data(ticker.split(".")[0], start_date=start, end_date=end)[entry]
    except: 
        try:
            raw = get_data(ticker.replace(".","-"), start_date=start, end_date=end)[entry]
        except:
            raw = get_data(ticker[0:4], start_date=start, end_date=end)[entry]
    return raw/raw[0]

def get_df(l,start,end,entry="adjclose"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        x=[executor.submit(lambda x: get_seiries(x,start,end,entry=entry),name)for name in l]
        x=[a.result() for a in x]
    for i,c in enumerate(x):
        c.name=l[i]
    return pd.DataFrame(x).transpose()

def tester_func(name,start=start,end=end,entry="adjclose"):
    try:
        c= get_seiries(name,start,end,entry)
        c.name=name
        return (True,c)
    except:
        return (False,name)

In [6]:
mini=ticker_list
errors=[]
data=[]
c=0
x=list(range(len(mini)))
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(len(mini)):
        x[i]=executor.submit(tester_func,mini[i])
    
    print("thread setup complete")
    
    for i in range(len(mini)):
        if(i%100==0):
            print(i)
        r=x[i].result()
        if (r[0]): 
            data.append(r[1])
        else:
            errors.append(r[1])
            c+=1
            print(f"error number {c}: name={errors[-1]}")

print("done")

thread setup complete
0
100
200
300
error number 1: name=KDIV
400
500
600
700
error number 2: name=OAIM
800
error number 3: name=CTEST.O
error number 4: name=EAI
900
1000
1100
1200
1300
1400
1500
error number 5: name=ZBZX
1600
error number 6: name=CTEST.E
1700
error number 7: name=ZJZZT
error number 8: name=XTWY
1800
error number 9: name=ZXIET
error number 10: name=ATEST.B
1900
2000
error number 11: name=RCA
2100
error number 12: name=ZVZZT
error number 13: name=LNKB
2200
2300
2400
2500
2600
error number 14: name=CBX
error number 15: name=GGLS
error number 16: name=ATEST
2700
error number 17: name=IBO
2800
error number 18: name=ZBZZT
2900
error number 19: name=ZAZZT
3000
error number 20: name=ZCZZT
3100
3200
3300
3400
error number 21: name=ATEST.A
3500
3600
3700
error number 22: name=ZEXIT
3800
3900
4000
error number 23: name=DHCNL
4100
error number 24: name=ZXZZT
4200
error number 25: name=CBO
4300
error number 26: name=IBIT
4400
error number 27: name=ZXYZ.A
4500
4600
error number 28:

In [7]:
data_df=pd.DataFrame(data).transpose()
data_df

,GRES,ONTO,NOGN,COWNL,SIZE,HEPS,DXJ,FRBN,REX,PR,...,GOOD,IGC,OPFI,JPS,CNDT,APYX,KRBN,GCV,FARM,WBIG
2020-01-02,1.000000,1.000000,NaN,1.000000,1.000000,NaN,1.000000,NaN,1.000000,1.000000,...,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000
2020-01-03,0.995231,0.985957,NaN,1.000731,0.995295,NaN,0.982692,NaN,1.003268,1.021322,...,1.004701,1.000000,NaN,1.004004,0.987603,0.995300,NaN,1.008532,0.992476,0.993623
2020-01-06,0.997065,0.970564,NaN,1.000548,0.996727,NaN,0.990241,NaN,0.993101,1.068230,...,1.004231,0.984127,NaN,1.001001,0.923967,0.974148,NaN,1.005120,0.996580,0.992526
2020-01-07,0.993397,0.976236,NaN,0.996894,0.996216,NaN,0.990425,NaN,0.962479,1.066098,...,0.989657,0.984127,NaN,1.004004,0.945454,0.971798,NaN,1.006826,0.967852,0.989643
2020-01-08,0.989362,0.992709,NaN,1.002558,0.999489,NaN,0.995949,NaN,0.935004,0.982942,...,0.991067,1.015873,NaN,1.004004,0.950413,0.967097,NaN,1.006826,0.941861,0.993095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,1.259090,1.821766,0.202832,1.134360,1.242030,0.073716,1.268203,1.022312,1.027596,1.611940,...,1.018092,0.873016,0.265990,0.841365,0.669421,0.702703,1.948549,1.136384,0.343365,1.064245
2022-09-08,1.262720,1.890359,0.171704,1.133779,1.254682,0.077439,1.286874,1.022312,1.047930,1.656716,...,1.019169,0.888889,0.265990,0.835586,0.659504,0.707403,1.895037,1.124165,0.348153,1.064783
2022-09-09,1.298613,1.930597,0.156642,1.139096,1.276690,0.078183,1.297093,1.022312,1.062455,1.692964,...,1.027783,0.873016,0.268020,0.836742,0.674380,0.735605,1.894055,1.138421,0.374829,1.073552
2022-09-12,1.311922,1.944910,0.151622,1.133466,1.287960,0.083395,1.295914,1.022312,1.064633,1.752665,...,1.036936,0.920635,0.275127,0.841365,0.692562,0.781434,2.017772,1.142494,0.372093,1.074628


In [349]:
train_df=data_df.iloc[:-68]
test_df=data_df.iloc[-68:]

test_df

,GRES,ONTO,NOGN,COWNL,SIZE,HEPS,DXJ,FRBN,REX,PR,...,GOOD,IGC,OPFI,JPS,CNDT,APYX,KRBN,GCV,FARM,WBIG
2022-06-07,1.452672,2.259519,1.006125,1.133254,1.303582,0.074460,1.295589,1.006085,1.149843,2.040512,...,1.081622,0.666667,0.322843,0.882971,0.912397,0.853114,2.352104,1.213772,0.350889,1.125753
2022-06-08,1.437347,2.178774,1.006125,1.120102,1.287057,0.074460,1.291705,1.006085,1.133866,1.997868,...,1.062240,1.301587,0.354315,0.886438,0.890909,0.836663,2.356522,1.211736,0.346785,1.116860
2022-06-09,1.403873,2.113962,1.005121,1.109142,1.253619,0.070737,1.283938,1.006085,1.151779,2.017058,...,1.040166,0.984127,0.332995,0.878348,0.867769,0.881316,2.339339,1.199516,0.350205,1.106767
2022-06-10,1.378062,2.019984,1.006125,1.098182,1.216088,0.067014,1.260830,1.006085,1.120673,1.972281,...,1.037474,0.825397,0.338071,0.862168,0.819835,0.733255,2.353577,1.230064,0.344049,1.097791
2022-06-13,1.324021,1.941399,1.004117,1.097174,1.162290,0.057930,1.237140,1.014199,1.035706,1.846482,...,0.998710,0.761905,0.315736,0.835586,0.737190,0.730905,2.291227,1.124165,0.329685,1.077440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-07,1.259090,1.821766,0.202832,1.134360,1.242030,0.073716,1.268203,1.022312,1.027596,1.611940,...,1.018092,0.873016,0.265990,0.841365,0.669421,0.702703,1.948549,1.136384,0.343365,1.064245
2022-09-08,1.262720,1.890359,0.171704,1.133779,1.254682,0.077439,1.286874,1.022312,1.047930,1.656716,...,1.019169,0.888889,0.265990,0.835586,0.659504,0.707403,1.895037,1.124165,0.348153,1.064783
2022-09-09,1.298613,1.930597,0.156642,1.139096,1.276690,0.078183,1.297093,1.022312,1.062455,1.692964,...,1.027783,0.873016,0.268020,0.836742,0.674380,0.735605,1.894055,1.138421,0.374829,1.073552
2022-09-12,1.311922,1.944910,0.151622,1.133466,1.287960,0.083395,1.295914,1.022312,1.064633,1.752665,...,1.036936,0.920635,0.275127,0.841365,0.692562,0.781434,2.017772,1.142494,0.372093,1.074628


In [350]:
ticker_list=data_df.columns
print(f"{len(symbols)-len(ticker_list)} stocks were droped")

1644 stocks were droped


# preprocessing 


1. ticker: this is also used to select which stocks come to the model

2. price: we take the ratio: (importantly we did drop some potentialy important indicators)

3. date: the day and month are floating point numbers. while year and day of the week are embedings.



In [351]:
look=layers.StringLookup(vocabulary=ticker_list,num_oov_indices=0)
inv_look=layers.StringLookup(vocabulary=ticker_list,invert=True,num_oov_indices=0)
tick_embed=layers.Embedding(len(ticker_list)+1,512,embeddings_constraint= tf.keras.constraints.unit_norm)

In [407]:
inputs=layers.Input([],dtype=tf.string)
x=look(inputs)
x=tick_embed(x)
ticker_model=tf.keras.Model(inputs,x)

In [408]:
dataset=tf.data.Dataset.from_tensor_slices(ticker_list)
exmple=dataset.batch(17).take(1).get_single_element()
exmple

<tf.Tensor: shape=(17,), dtype=string, numpy=
array([b'GRES', b'ONTO', b'NOGN', b'COWNL', b'SIZE', b'HEPS', b'DXJ',
       b'FRBN', b'REX', b'PR', b'MDWD', b'LEVI', b'HMC', b'AIKI', b'SPUC',
       b'FANG', b'SAVN'], dtype=object)>

In [409]:
#data_df[exmple.numpy().astype('U13')]

In [410]:
def thread_distribute(l,f,n=1):
    if(n==0):
        return f(l)
    
    with concurrent.futures.ThreadPoolExecutor() as executor: 
        l=[executor.submit(lambda x: thread_distribute(x,f,n-1),x) for x in l]
        l= [x.result() for x in l]
    return l

thread_distribute([range(5) for i in range(2)],lambda x: range(x),2)

[[range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)],
 [range(0, 0), range(0, 1), range(0, 2), range(0, 3), range(0, 4)]]

In [496]:
def choose_stocks(x,data_df=train_df):
    #a=tick_embed(look(ticker_list))
    #b=tick_embed(look(x))
    a=ticker_model(ticker_list)
    b=ticker_model(x)
    
    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,NUM_STOCKS)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    return [[data_df[x] for x in sub] for sub in tickers]
    #return thread_distribute(tickers,lambda x: data_df[x],2)

a=choose_stocks(exmple)

pd.DataFrame(a[0]).transpose().dropna()

,GRES,HIMS
2020-01-02,1.000000,1.000000
2020-01-03,0.995231,1.000000
2020-01-06,0.997065,1.000000
2020-01-07,0.993397,1.002020
2020-01-08,0.989362,1.003030
...,...,...
2022-05-31,1.416375,0.408081
2022-06-01,1.414359,0.407071
2022-06-02,1.434927,0.429293
2022-06-03,1.429281,0.410101


In [497]:
a[0][0].index

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13',
               '2020-01-14', '2020-01-15',
               ...
               '2022-05-23', '2022-05-24', '2022-05-25', '2022-05-26',
               '2022-05-27', '2022-05-31', '2022-06-01', '2022-06-02',
               '2022-06-03', '2022-06-06'],
              dtype='datetime64[ns]', length=612, freq=None)

In [530]:
def time_encode(a):
    #a=a.index
    return (a.day_of_week,a.day/30,a.month/12,(a.year-2020))
    
time_encode(data_df.index[0])

(3, 0.06666666666666667, 0.08333333333333333, 0)

In [499]:
def time_tensors(a):
    x=thread_distribute(a,lambda x:time_encode(x.index),2)
    x=[thread_distribute(x,lambda x: x[i],2) for i in range(4)]
    return [tf.constant(x) for x in x]

time_tensors(a)

[<tf.Tensor: shape=(17, 2, 612), dtype=int32, numpy=
 array([[[3, 4, 0, ..., 3, 4, 0],
         [3, 4, 0, ..., 3, 4, 0]],
 
        [[3, 4, 0, ..., 3, 4, 0],
         [3, 4, 0, ..., 3, 4, 0]],
 
        [[3, 4, 0, ..., 3, 4, 0],
         [3, 4, 0, ..., 3, 4, 0]],
 
        ...,
 
        [[3, 4, 0, ..., 3, 4, 0],
         [3, 4, 0, ..., 3, 4, 0]],
 
        [[3, 4, 0, ..., 3, 4, 0],
         [3, 4, 0, ..., 3, 4, 0]],
 
        [[3, 4, 0, ..., 3, 4, 0],
         [3, 4, 0, ..., 3, 4, 0]]])>,
 <tf.Tensor: shape=(17, 2, 612), dtype=float64, numpy=
 array([[[0.06666667, 0.1       , 0.2       , ..., 0.06666667,
          0.1       , 0.2       ],
         [0.06666667, 0.1       , 0.2       , ..., 0.06666667,
          0.1       , 0.2       ]],
 
        [[0.06666667, 0.1       , 0.2       , ..., 0.06666667,
          0.1       , 0.2       ],
         [0.06666667, 0.1       , 0.2       , ..., 0.06666667,
          0.1       , 0.2       ]],
 
        [[0.06666667, 0.1       , 0.2       , ..., 0

# time handeling 

problems with the data in curent time format:

1. there are null entries especialy from before a company was made 
2. the time diffrence between entries isnt clean

In [500]:
mask=np.logical_not(data_df.isna().to_numpy())
mask

array([[ True,  True, False, ...,  True,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       [ True,  True, False, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [527]:
def choose_stocks_df(x,data_df=train_df):
    #a=tick_embed(look(ticker_list))
    #b=tick_embed(look(x))
    a=ticker_model(ticker_list)
    b=ticker_model(x)

    b=b@tf.transpose(a)
    b=tf.nn.top_k(b,NUM_STOCKS)


    tickers=inv_look([list(x) for x in b[1].numpy()]).numpy().astype('U13')
    
    
    return [pd.DataFrame([data_df[x] for x in sub]).transpose().dropna() for sub in tickers]

v=choose_stocks_df(exmple)
v[0]

,GRES,HIMS
2020-01-02,1.000000,1.000000
2020-01-03,0.995231,1.000000
2020-01-06,0.997065,1.000000
2020-01-07,0.993397,1.002020
2020-01-08,0.989362,1.003030
...,...,...
2022-05-31,1.416375,0.408081
2022-06-01,1.414359,0.407071
2022-06-02,1.434927,0.429293
2022-06-03,1.429281,0.410101


In [552]:
# need to add the tickers strings/ints
def get_xs(v):
    t=[x.index for x in v]
    for x in t:
        print(type(x))
    t=thread_distribute(t,time_encode,2)
    #t=[[x[i] for x in t] for i in range(4)]
    t=[thread_distribute(t,lambda x: x[i],2) for i in range(4)]
    #thread_distribute(t,len,2)
    t=[tf.ragged.constant(x) for x in t]
    
    x=thread_distribute(v,lambda x: x.to_numpy(),1)
    vals=tf.concat(x,axis=0)
    lengths=[len(s) for s in x]
    x=tf.RaggedTensor.from_row_lengths(vals,lengths)
    #[x.shape for x in x]
    
    "padding"
    x=(x+1).to_tensor()-1
    t=[(x+1).to_tensor()-1 for x in t] 
    
    x=tf.transpose(x,[2,0,1])
    t.extend(x) 
    
    tickers=tf.constant([x.transpose().index for x in v])
    t.append(tickers)
    return t

xs=get_xs(v)
[(x.shape,x.dtype) for x in xs]

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


TypeError: 'KerasTensor' object is not callable

In [503]:
#[len(x) for x in v]

In [504]:
def sample_seq(x,LEN=100):
    l=len(x)
    if(l<LEN):
        return x 
    
    start=np.random.randint(l-LEN)
    return x.iloc[start:start+LEN]
sample_seq(v[-1]) 
sample_seq(v[0])

,GRES,HIMS
2020-05-01,0.865737,1.008081
2020-05-04,0.868672,1.008081
2020-05-05,0.874175,1.010101
2020-05-06,0.866838,1.009091
2020-05-07,0.876376,1.009091
...,...,...
2020-09-16,0.971387,1.131313
2020-09-17,0.974321,1.131313
2020-09-18,0.970653,1.130303
2020-09-21,0.950477,1.110101


In [520]:
base=[sample_seq(x) for x in v]
y=[s.iloc[1:,0] for s in base]
x=[s[:-1] for s in base]
x=get_xs(x)
x

TypeError: 'KerasTensor' object is not callable

In [548]:
def get_numbers(tickers):
    v=choose_stocks_df(tickers)
    base=[sample_seq(x) for x in v]
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    #x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_numbers(exmple)
print(time.time()-t)

TypeError: 'KerasTensor' object is not callable

In [459]:
specs=[tf.TensorSpec.from_tensor(x) for x in n]
s=[list(x.shape) for x in specs]
for x in s:
    x[0]=None
specs=[tf.TensorSpec(s[i],specs[i].dtype) for i in range(len(specs))]
specs

[TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.string, name=None),
 TensorSpec(shape=(None, 99), dtype=tf.float64, name=None)]

In [460]:
def gather_x(*args):
    return args[:-1]

def gather_y(*args):
    return args[-1]

gather_x(1,2,3,4),gather_y(1,2,3,4)

((1, 2, 3), 4)

# train dataset 
because of the need to use the dataset as a refrence type in the pandas df we are forcing tensorflow to run the whole thing eagerly. 

it shouldnt be a major issue since most the heavy lifting is done in pandas which is optimized c code 

In [461]:
#need to do training and test set (test set should look diffrent)
train_dataset=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_numbers,inp=[y],Tout=specs))

dataset_x=train_dataset.map(gather_x)
dataset_y=train_dataset.map(gather_y)
train_dataset=tf.data.Dataset.zip((dataset_x,dataset_y))

x=train_dataset.take(1).get_single_element()
x

((<tf.Tensor: shape=(32, 99), dtype=int32, numpy=
  array([[4, 0, 1, ..., 2, 3, 4],
         [0, 1, 2, ..., 0, 1, 2],
         [2, 3, 4, ..., 1, 2, 3],
         ...,
         [0, 1, 2, ..., 3, 4, 0],
         [2, 3, 4, ..., 1, 2, 3],
         [2, 3, 4, ..., 4, 0, 1]])>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  array([[1.0333333 , 0.10000002, 0.13333333, ..., 0.5333333 , 0.5666666 ,
          0.6       ],
         [0.36666667, 0.39999998, 0.4333334 , ..., 0.9333334 , 0.0333333 ,
          0.06666672],
         [0.9666667 , 1.        , 0.0333333 , ..., 0.5       , 0.5333333 ,
          0.5666666 ],
         ...,
         [0.23333335, 0.26666665, 0.29999995, ..., 0.70000005, 0.73333335,
          0.83333325],
         [0.36666667, 0.39999998, 0.4333334 , ..., 0.9333334 , 0.9666667 ,
          1.        ],
         [0.23333335, 0.26666665, 0.29999995, ..., 0.6333333 , 0.73333335,
          0.76666665]], dtype=float32)>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  arr

In [425]:
[tf.TensorSpec.from_tensor(x) for x in x[0]]

[TensorSpec(shape=(32, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.int32, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 99), dtype=tf.float64, name=None),
 TensorSpec(shape=(32, 2), dtype=tf.string, name=None)]

In [427]:
def cut23(x):
    return {"time":x[:4],"prices":x[4:NUM_STOCKS],"tickers":x[NUM_STOCKS]}

cut23(x[0])

{'time': (<tf.Tensor: shape=(32, 99), dtype=int32, numpy=
  array([[ 1,  2,  3, ...,  0,  1,  2],
         [ 1,  2,  3, ...,  1,  2,  3],
         [ 1,  2,  3, ...,  1,  2,  3],
         ...,
         [ 4,  0,  1, ..., -1, -1, -1],
         [ 1,  2,  3, ...,  1,  2,  3],
         [ 0,  1,  2, ..., -1, -1, -1]])>,
  <tf.Tensor: shape=(32, 99), dtype=float32, numpy=
  array([[ 0.10000002,  0.13333333,  0.16666663, ...,  0.66666675,
           0.70000005,  0.73333335],
         [ 0.23333335,  0.26666665,  0.29999995, ...,  0.8666667 ,
           0.9       ,  0.9333334 ],
         [ 0.29999995,  0.33333337,  0.36666667, ...,  0.9666667 ,
           1.        ,  1.0333333 ],
         ...,
         [ 0.13333333,  0.23333335,  0.26666665, ..., -1.        ,
          -1.        , -1.        ],
         [ 0.70000005,  0.73333335,  0.76666665, ...,  0.33333337,
           0.36666667,  0.39999998],
         [ 0.76666665,  0.79999995,  0.83333325, ..., -1.        ,
          -1.        , -1.      

# validation dataset 
on infrance the best results would probably be achived by runing on every time entry with at least some memory of the past

that means there would need to be some precomputation which we can make diffrent for timesetep. 

this is rediclously slow and combersom to write so our validation is just gona look mostly like the training. 
after choosing the stocks we would want to work with we are going to think of test behivior 

we may want to just train a new model from scratch only on that subset.

In [428]:
def get_val_numbers(tickers):
    base=choose_stocks_df(tickers,test_df)
    
    y=[s.iloc[1:,0].to_numpy() for s in base]
    y=tf.ragged.constant(y)
    y=(y+1).to_tensor()-1
    
    x=[s[:-1] for s in base]
    x=get_xs(x)
    #x.append(tickers)
    x.append(y)
    return x
t=time.time()
n=get_val_numbers(exmple)
print(time.time()-t)

1.623509407043457


In [429]:
#need to do training and test set (test set should look diffrent)
val_dataset=dataset.shuffle(1000).batch(32).map(lambda y: tf.py_function(get_val_numbers,inp=[y],Tout=specs))

dataset_x=val_dataset.map(gather_x)
dataset_y=val_dataset.map(gather_y)
val_dataset=tf.data.Dataset.zip((dataset_x,dataset_y))

x=val_dataset.take(1).get_single_element()

In [537]:
[tf.TensorSpec.from_tensor(x) for x in x[0]]

[TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 TensorSpec(shape=(99,), dtype=tf.int32, name=None)]

# the model

**the dataset is dependent on the model!!!** this means we have some considerations on how it should look like:

* there has to be an attention mechanisem on embeding space. (using only the sampling embedings dot product) 
* embedings convergnce and stabilety greatly effects the models training. (we may want to consider reverse regularization)
*  at least some part of the model should be seprable by stock, in order to reduce noise from data sampeling
* it would defintly help if we add residual conections to the embedings to alow gradients to flow to the embedings

*side note: the padding used is -1, seems like it should propgate to the loss*


In [538]:
#setting up the inputs
inputs=[layers.Input([None,],dtype=x.dtype) for x in specs[:-2]]
inputs.append(layers.Input(specs[-2].shape[1:],dtype=specs[-2].dtype,name="tickers"))

#time encoding
time_depndent=[tf.expand_dims(x,-1) for x in inputs[:-1]] 
time_depndent=[layers.Masking(-1)(x) for x in time_depndent] 


day_embeding=layers.Embedding(7,7)(time_depndent[0])
year_embeding=layers.Embedding(3,5)(time_depndent[3]) 

day_embeding=layers.TimeDistributed(layers.Reshape([7]))(day_embeding)
year_embeding=layers.TimeDistributed(layers.Reshape([5]))(year_embeding)

t=tf.concat(time_depndent[1:3],-1)
t=layers.Dense(20)(t) 

time_encode=tf.concat([day_embeding,t,year_embeding],-1)
time_encode

<KerasTensor: shape=(None, None, 32) dtype=float32 (created by layer 'tf.concat_87')>

In [539]:
#tickers=look(inputs[-1])
#tickers=tick_embed(tickers)
tickers=ticker_model(inputs[-1])
origin=tf.expand_dims(tickers[:,0],-2)
origin=layers.Permute([2,1])(origin)
attention=tickers@origin
attention=layers.Permute([2,1])(attention)
attention=tf.nn.softmax(attention)
attention

<KerasTensor: shape=(None, 1, 2) dtype=float32 (created by layer 'tf.nn.softmax_12')>

In [540]:
comp=layers.Dense(33)(tickers)
comp

<KerasTensor: shape=(None, 2, 33) dtype=float32 (created by layer 'dense_30')>

In [541]:
"need to add stock conditioning"
prices=time_depndent[4:4+NUM_STOCKS]
s=[layers.Concatenate(-1)([time_encode,x]) for x in prices]

#adding ticker information (while not alowing it to effect the mask)
masks=[x._keras_mask for x in s] 
s=[layers.Add()([s[i],comp[:,i]]) for i in range(len(s))]

stock_layer=layers.LSTM(16,return_sequences=True)
s=[stock_layer(s[i],mask=masks[i]) for i in range(len(s))]
stock_layer=layers.GRU(16,return_sequences=True)
s=[stock_layer(x) for x in s]

s=[tf.expand_dims(x,-2) for x in s]
s=layers.Concatenate(-2)(s)
s

<KerasTensor: shape=(None, None, 2, 16) dtype=float32 (created by layer 'concatenate_68')>

In [542]:
sumorized=attention@tf.transpose(s,[1,0,2,3])
sumorized=tf.transpose(sumorized,[1,0,2,3])
sumorized=layers.TimeDistributed(layers.Reshape([16]))(sumorized)
sumorized

<KerasTensor: shape=(None, None, 16) dtype=float32 (created by layer 'time_distributed_71')>

In [543]:
x=layers.LSTM(8,return_sequences=True,)(sumorized)
x=layers.LayerNormalization()(x)
x=layers.Permute([2,1])(x)
x=layers.GlobalMaxPool1D()(x)
x

<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'global_max_pooling1d_11')>

In [544]:
model=tf.keras.Model(inputs,x)
model.compile("adam","mse")
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_326 (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 input_327 (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 input_328 (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 input_329 (InputLayer)         [(None, None)]       0           []                               
                                                                                           

 licingOpLambda)                                                                                  
                                                                                                  
 add_39 (Add)                   (None, None, 33)     0           ['concatenate_66[0][0]',         
                                                                  'tf.__operators__.getitem_69[0][
                                                                 0]']                             
                                                                                                  
 add_40 (Add)                   (None, None, 33)     0           ['concatenate_67[0][0]',         
                                                                  'tf.__operators__.getitem_70[0][
                                                                 0]']                             
                                                                                                  
 tf.expand

In [547]:
numbers=get_numbers(exmple)
x=numbers[:-1]
y=numbers[-1]

TypeError: 'KerasTensor' object is not callable

In [ ]:
model.train_on_batch(x,y)

In [469]:
a=tf.ones([17,1,2])
b=tf.ones([99,17,2,16])

a@b

<tf.Tensor: shape=(99, 17, 1, 16), dtype=float32, numpy=
array([[[[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        ...,

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]]],


       [[[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        ...,

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]]],


       [[[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        ...,

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]]],


       ...,


       [[[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.]],


ValueError: as_list() is not defined on an unknown TensorShape.